# 🌱 什么是迁移学习？

迁移学习是一种 `利用已有模型的知识来解决新任务` 的方法。通常用在 `数据量较少的新任务` 上，比如你只有几千张猫狗图片，但你可以利用在 `ImageNet` 上训练过的 大规模 `CNN` 模型（如 `ResNet`、`MobileNet`）来加快训练和提高准确率。

- 核心思想：

    - 不从零开始训练（避免过拟合、减少算力需求）。
    
    - 复用大模型学到的 特征提取能力。

---
# 🔑 迁移学习常见方式

- 1.特征提取 (Feature Extraction)

    - 冻结预训练模型的卷积层，只在输出层上添加新的分类器并训练。
    
    - 适合数据量少、和原任务相似的场景。

- 2.微调 (Fine-tuning)

    - 解冻部分预训练层（通常是靠后的层），连同新分类层一起训练。
    
    - 适合数据量中等或任务和预训练任务有一定差异时。
---

## ⚙️ TensorFlow/Keras 实现迁移学习
### 1. 加载预训练模型

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 以 MobileNetV2 为例（ImageNet 预训练）
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,  # 不要最后的全连接层
    weights='imagenet'  
)

# 冻结预训练层
base_model.trainable = False 

1.
`input_shape`=(224,224,3)
    - 指定输入图像的尺寸为224×224像素，3个颜色通道（RGB）
    - 这决定了网络接受的输入张量的形状

2.
`include_top` = `False`
    - 不包含模型的顶层（全连接分类层）
    - 通常用于迁移学习，这样我们可以添加自己的分类层
    - 当设为`False`时，输出将是特征图而不是类别预测

3.
`weights`= `'imagenet'`
    - 使用在ImageNet数据集上预训练的权重初始化模型
    - 这使得模型已经具备从图像中提取有用特征的能力
    - 如果不设置或设为None，模型将使用随机初始化的权重

4.
`bast_model.trainable` = `False`

    - 冻结基础模型的所有层，使其在训练过程中不可训练
    - 这意味着在`后续训练`中只会`更新新添加的层`的权重
    - 这可以防止预训练的特征提取能力被破坏，同时加快训练速度
---
### 2. 冻结卷积基（特征提取）

In [ ]:
 
# 搭建新模型
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # 全局平均池化
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # 假设有 10 个类别
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


### 3. 训练分类器

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=5)


### 4. 微调部分层

当`分类层训练好`后，可以`解冻`一部分卷积层进行 `微调`：

In [ ]:
# 解冻部分层（例如最后 20 层）
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# 使用较小学习率继续训练
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(train_ds, validation_data=val_ds, epochs=5)


## 📊 迁移学习的优点

 - 减少训练时间。

 - 提高小数据集上的表现。

 - 避免过拟合。

---
## 🚀 推荐的预训练模型（tf.keras.applications）

 - `VGG16` / `VGG19`

 - `ResNet50`

 - `InceptionV3`

 - `MobileNetV2`（轻量级，适合移动端）

 - `EfficientNetB0~B7`（性能最优）

# 完整的猫狗分类迁移学习项目：
数据准备 → 模型构建 → 训练 → 可视化结果

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# 下载数据集
import tensorflow_datasets as tfds

(train_ds, val_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,  # 返回 (image, label)
    with_info=True
)

# 预处理函数
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))   # 调整到模型输入大小
    image = tf.cast(image, tf.float32) / 255.0   # 归一化
    return image, label

# 应用预处理 + 批处理
BATCH_SIZE = 32
train_ds = train_ds.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# 加载预训练模型（不含顶层）
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# 冻结卷积基
base_model.trainable = False

# 搭建完整模型
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # 二分类
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=5)
# 解冻部分卷积层
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=3)

def plot_history(history, fine_tune_history=None):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='train_acc')
    plt.plot(val_acc, label='val_acc')
    plt.legend()
    plt.title("Accuracy")

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='train_loss')
    plt.plot(val_loss, label='val_loss')
    plt.legend()
    plt.title("Loss")

    plt.show()

plot_history(history)

## 如果加了微调
#plot_history(history, history_fine)